In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')

In [2]:
%matplotlib nbagg
import numpy as np
import json
import os
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

from alvra_tools.load_data import *
from alvra_tools.channels import *
from alvra_tools.utils import errfunc_sigma, errfunc_fwhm
from built_functions import *

[INFO][corrections] Numba not available, reverting to Numpy


Set up the filters.  "numstds" is the number of standard deviations to take from the median.

Plot some diagnostics

In [3]:
def get_XAS_timescans(scan_name, DIR, DIR_json, numstds, minIzero, lin_filter):


    json_file = DIR_json + scan_name + "_scan_info.json"
    print (json_file)

    with open(json_file) as file:
        data = json.load(file)
    numFiles = len(data['scan_files'])

    delay_mm = np.empty(0)
    iZero = np.empty(0)
    iZero = np.empty(0)
    
    err_DataFluo_pump_total = np.empty(0)
    err_DataFluo_unpump_total = np.empty(0)

    
    DataFluo_pump_norm_total = np.empty(0)
    DataFluo_unpump_norm_total = np.empty(0)

    IzeroFEL_pump_original_total = np.empty(0)
    IzeroFEL_unpump_original_total = np.empty(0)    
    DataFluo_pump_original_total = np.empty(0)
    DataFluo_unpump_original_total = np.empty(0)
    
    IzeroFEL_pump_total = np.empty(0)
    IzeroFEL_unpump_total = np.empty(0)
    DataFluo_pump_total = np.empty(0)
    DataFluo_unpump_total = np.empty(0)
    
    for i in range(0,numFiles):
        filename = str(data['scan_files'][i][0])
        filename = DIR + os.path.basename(filename)
        exists = os.path.isfile(filename)
        if not exists:
            print("No such file")
        elif exists: #Do we need this 39?
            (DataFluo_pump, DataFluo_unpump, IzeroFEL_pump, IzeroFEL_unpump, delay, _, _, _) = \
                load_PumpProbe_events(filename, channel_delay_SH)
                
#             print('\n\n\n'+scan_name)
#             print('shape of pumped data')
#             print(DataFluo_pump.shape)
#             print('shape of unpumped data')
#             print(DataFluo_unpump.shape)
                
            IzeroFEL_pump_original = IzeroFEL_pump.copy()
            IzeroFEL_unpump_original = IzeroFEL_unpump.copy()

            IzeroFEL_pump_original_total = np.append(IzeroFEL_pump_original_total, IzeroFEL_pump_original)
            IzeroFEL_unpump_original_total = np.append(IzeroFEL_unpump_original_total, IzeroFEL_unpump_original)

            DataFluo_pump_original_total = np.append(DataFluo_pump_original_total, DataFluo_pump)
            DataFluo_unpump_original_total = np.append(DataFluo_unpump_original_total, DataFluo_unpump)
            
            # filter
            IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, condFinalPump, condFinalUnPump\
                =filterData(IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, numstds, minIzero, lin_filter)    

            DataFluo_pump_norm = DataFluo_pump/IzeroFEL_pump
            DataFluo_unpump_norm = DataFluo_unpump/IzeroFEL_unpump

            DataFluo_pump_norm_total = np.append(DataFluo_pump_norm_total, DataFluo_pump_norm.mean())
            DataFluo_unpump_norm_total = np.append(DataFluo_unpump_norm_total, DataFluo_unpump_norm.mean())
            
            IzeroFEL_pump_total = np.append(IzeroFEL_pump_total, IzeroFEL_pump)
            IzeroFEL_unpump_total = np.append(IzeroFEL_unpump_total, IzeroFEL_unpump)

            
            err_DataFluo_pump_total = np.append(err_DataFluo_pump_total, DataFluo_pump_norm.std()/np.sqrt(DataFluo_pump_norm.size))
            err_DataFluo_unpump_total = np.append(err_DataFluo_unpump_total, DataFluo_unpump_norm.std()/np.sqrt(DataFluo_unpump_norm.size))
          
            delay = [x for x in delay if (np.abs(x) > 0)]
            delay_mm = np.append(delay_mm, np.mean(delay))
            iZero = [x for x in iZero if (np.abs(x) > 0)]
            iZero = np.append(iZero, np.mean(IzeroFEL_pump_total))            


    return iZero, delay_mm, DataFluo_unpump_norm_total, DataFluo_pump_norm_total, err_DataFluo_unpump_total, err_DataFluo_pump_total
        

In [4]:
# def get_XAS(scan_name, DIR, DIR_json, numstds, minIzero):
#     #numstds = 2
#     #minIzero = 0.015

#     json_file = DIR_json + scan_name + "_scan_info.json"
#     print (json_file)

#     with open(json_file) as file:
#         data = json.load(file)
#     numFiles = len(data['scan_files'])

#     print ("Processing",numFiles,"files")

#     DataFluo_pump_norm_total = np.empty(0)
#     DataFluo_unpump_norm_total = np.empty(0)
#     err_DataFluo_pump_total = np.empty(0)
#     err_DataFluo_unpump_total = np.empty(0)

#     IzeroFEL_pump_original_total = np.empty(0)
#     IzeroFEL_pump_total = np.empty(0)
#     IzeroFEL_unpump_original_total = np.empty(0)
#     IzeroFEL_unpump_total = np.empty(0)

#     DataFluo_pump_original_total = np.empty(0)
#     DataFluo_pump_total = np.empty(0)
#     DataFluo_unpump_original_total = np.empty(0)
#     DataFluo_unpump_total = np.empty(0)

#     delay_mm = np.empty(0)
#     iZero = np.empty(0)
    

#     for i in range(0,numFiles):
#     #for i in range(0,1):
#         filename = str(data['scan_files'][i][0])
#         filename = DIR + os.path.basename(filename)
#         exists = os.path.isfile(filename)
#         if not exists:
#             print("No such file")
#         elif exists: #Do we need this 39?
# #             print("step",i+1,"of",numFiles,": Processing %s" %(str(data['scan_files'][i][0])))
#             (DataFluo_pump, DataFluo_unpump, IzeroFEL_pump, IzeroFEL_unpump, delay, _, _, _) = \
#                 load_PumpProbe_events(filename, channel_delay_SH)

#             IzeroFEL_pump_original = IzeroFEL_pump.copy()
#             IzeroFEL_unpump_original = IzeroFEL_unpump.copy()

#             IzeroFEL_pump_original_total = np.append(IzeroFEL_pump_original_total, IzeroFEL_pump_original)
#             IzeroFEL_unpump_original_total = np.append(IzeroFEL_unpump_original_total, IzeroFEL_unpump_original)

#             DataFluo_pump_original_total = np.append(DataFluo_pump_original_total, DataFluo_pump)
#             DataFluo_unpump_original_total = np.append(DataFluo_unpump_original_total, DataFluo_unpump)

#             IzeroMedian = np.median(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))
#             IzeroSTD = np.std(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))

#             conditionPumpMax = IzeroFEL_pump < IzeroMedian+numstds*IzeroSTD
#             conditionPumpMin = IzeroFEL_pump > IzeroMedian-numstds*IzeroSTD
#             conditionPumpLow = IzeroFEL_pump > minIzero

#             conditionUnPumpMax = IzeroFEL_unpump < IzeroMedian+numstds*IzeroSTD
#             conditionUnPumpMin = IzeroFEL_unpump > IzeroMedian-numstds*IzeroSTD
#             conditionUnPumpLow = IzeroFEL_unpump > minIzero

#             condIzeroPump = conditionPumpMax & conditionPumpMin & conditionPumpLow
#             condIzeroUnPump = conditionUnPumpMax & conditionUnPumpMin & conditionUnPumpLow

#             IzeroFEL_pump = IzeroFEL_pump[condIzeroPump]
#             IzeroFEL_unpump = IzeroFEL_unpump[condIzeroUnPump]

#             IzeroFEL_pump_total = np.append(IzeroFEL_pump_total, IzeroFEL_pump)
#             IzeroFEL_unpump_total = np.append(IzeroFEL_unpump_total, IzeroFEL_unpump)

#             DataFluo_pump = DataFluo_pump[condIzeroPump]
#             DataFluo_unpump = DataFluo_unpump[condIzeroUnPump]

#             DataFluo_pump_total = np.append(DataFluo_pump_total, DataFluo_pump)
#             DataFluo_unpump_total = np.append(DataFluo_unpump_total, DataFluo_unpump)

#             DataFluo_pump_norm = DataFluo_pump/IzeroFEL_pump
#             DataFluo_unpump_norm = DataFluo_unpump/IzeroFEL_unpump

#             DataFluo_pump_norm_total = np.append(DataFluo_pump_norm_total, DataFluo_pump_norm.mean())
#             DataFluo_unpump_norm_total = np.append(DataFluo_unpump_norm_total, DataFluo_unpump_norm.mean())
#             err_DataFluo_pump_total = np.append(err_DataFluo_pump_total, DataFluo_pump_norm.std()/np.sqrt(DataFluo_pump_norm.size))
#             err_DataFluo_unpump_total = np.append(err_DataFluo_unpump_total, DataFluo_unpump_norm.std()/np.sqrt(DataFluo_unpump_norm.size))


            
#             delay = [x for x in delay if (np.abs(x) > 0)]
#             delay_mm = np.append(delay_mm, np.mean(delay))
#             iZero = [x for x in iZero if (np.abs(x) > 0)]
#             iZero = np.append(iZero, np.mean(IzeroFEL_pump_total))
            
#     plt.figure()
#     _, bins, _ = plt.hist(IzeroFEL_pump_original_total, 100, label = 'unfiltered')
#     _ = plt.hist(IzeroFEL_pump_total, bins, rwidth = .5, label = 'filtered')
#     plt.title('Izero, pumped')
#     plt.legend()

#     plt.figure()
#     _, bins, _ = plt.hist(IzeroFEL_unpump_original_total, 100, label = 'unfiltered')
#     _ = plt.hist(IzeroFEL_unpump_total, bins, rwidth = .5, label = 'filtered')
#     plt.title('Izero, unpumped')
#     plt.legend()
    
#     plt.figure()
#     plt.scatter(IzeroFEL_pump_original_total, DataFluo_pump_original_total)
#     plt.scatter(IzeroFEL_pump_total, DataFluo_pump_total)
#     plt.title('Izero, pumped')
#     plt.xlabel('I0')
#     plt.ylabel('Absorption')

#     delay_mm = delay_mm[1:]
#     DataFluo_pump_norm_total = DataFluo_pump_norm_total[1:]
#     DataFluo_unpump_norm_total = DataFluo_unpump_norm_total[1:]
#     iZero =iZero[1:]
    
#     plt.figure()
#     plt.plot(np.array(delay_mm))
    
#     plt.figure()
#     plt.plot(np.array(delay_mm), DataFluo_pump_norm_total,label='Pumped')
#     plt.plot(np.array(delay_mm), DataFluo_unpump_norm_total,label='UnPumped')
#     plt.plot(np.array(delay_mm), np.array(iZero)*100,label="I_zero_Arb")
#     plt.xlabel('delay (mm)')
#     plt.ylabel('absorption')
#     plt.title('XAS_'+ scan_name)
#     plt.legend()


#     return iZero, delay_mm, DataFluo_unpump_norm_total, DataFluo_pump_norm_total, err_DataFluo_unpump_total, err_DataFluo_pump_total
        

In [5]:
# def get_XAS(scan_name, DIR, DIR_json, numstds, minIzero, lin_filter):


#     json_file = DIR_json + scan_name + "_scan_info.json"
#     print (json_file)

#     with open(json_file) as file:
#         data = json.load(file)
#     numFiles = len(data['scan_files'])

#     Energy_eV = np.empty(0)
#     iZero = np.empty(0)
    
#     err_DataFluo_pump_total = np.empty(0)
#     err_DataFluo_unpump_total = np.empty(0)

    
#     DataFluo_pump_norm_total = np.empty(0)
#     DataFluo_unpump_norm_total = np.empty(0)

#     IzeroFEL_pump_original_total = np.empty(0)
#     IzeroFEL_unpump_original_total = np.empty(0)    
#     DataFluo_pump_original_total = np.empty(0)
#     DataFluo_unpump_original_total = np.empty(0)
    
#     IzeroFEL_pump_total = np.empty(0)
#     IzeroFEL_unpump_total = np.empty(0)
#     DataFluo_pump_total = np.empty(0)
#     DataFluo_unpump_total = np.empty(0)
    
#     for i in range(0,numFiles):
#         filename = str(data['scan_files'][i][0])
#         filename = DIR + os.path.basename(filename)
#         exists = os.path.isfile(filename)
#         if not exists:
#             print("No such file")
#         elif exists: #Do we need this 39?
#             (DataFluo_pump, DataFluo_unpump, IzeroFEL_pump, IzeroFEL_unpump, Energy, _, _, _) = \
#                 load_PumpProbe_events(filename, channel_energy)

#             IzeroFEL_pump_original = IzeroFEL_pump.copy()
#             IzeroFEL_unpump_original = IzeroFEL_unpump.copy()

#             IzeroFEL_pump_original_total = np.append(IzeroFEL_pump_original_total, IzeroFEL_pump_original)
#             IzeroFEL_unpump_original_total = np.append(IzeroFEL_unpump_original_total, IzeroFEL_unpump_original)

#             DataFluo_pump_original_total = np.append(DataFluo_pump_original_total, DataFluo_pump)
#             DataFluo_unpump_original_total = np.append(DataFluo_unpump_original_total, DataFluo_unpump)
            
#             # filter
#             IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, condFinalPump, condFinalUnPump\
#                 =filterData(IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, numstds, minIzero, lin_filter)    

#             DataFluo_pump_norm = DataFluo_pump/IzeroFEL_pump
#             DataFluo_unpump_norm = DataFluo_unpump/IzeroFEL_unpump

#             DataFluo_pump_norm_total = np.append(DataFluo_pump_norm_total, DataFluo_pump_norm.mean())
#             DataFluo_unpump_norm_total = np.append(DataFluo_unpump_norm_total, DataFluo_unpump_norm.mean())
            
#             IzeroFEL_pump_total = np.append(IzeroFEL_pump_total, IzeroFEL_pump)
#             IzeroFEL_unpump_total = np.append(IzeroFEL_unpump_total, IzeroFEL_unpump)
            
#             Energy = [x for x in Energy if (np.abs(x) > 0)]
#             Energy_eV = np.append(Energy_eV, np.mean(Energy))
            
#             iZero = [x for x in iZero if (np.abs(x) > 0)]
#             iZero = np.append(iZero, np.mean(IzeroFEL_pump_total))

#     return iZero, Energy_eV, IzeroFEL_pump_total, IzeroFEL_unpump_total, DataFluo_pump_norm_total, DataFluo_unpump_norm_total

        

In [6]:
def get_XAS_average_scanlist(numstds, minIzero, folder_name, scanlist, DIR_json):
    ploton = True


    iZero_all = np.empty(0)
    delay_mm_all = np.empty(0)
    delay_mm_all_err = np.empty(0)
    DataFluo_unpump_norm_total_all = np.empty(0)
    DataFluo_pump_norm_total_all = np.empty(0)
    DataFluo_unpump_norm_total_all_err = np.empty(0)
    DataFluo_pump_norm_total_all_err = np.empty(0)

    for jj in range(0, len(scanlist)):
        print('Scan # ' + str(scanlist[jj]))
        if scanlist[jj] < 10:
            scan_name = folder_name + '00' + str(scanlist[jj])
        else:
            scan_name = folder_name + '0' + str(scanlist[jj])

        DIR = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"    
        print('Opening ' + DIR + '...')

        (iZero, delay_mm, DataFluo_unpump_norm_total, DataFluo_pump_norm_total, err_DataFluo_unpump_total, err_DataFluo_pump_total)\
            = get_XAS_timescans(scan_name, DIR, DIR_json, numstds, minIzero, lin_filter)
            
        print('Shape of delays = ' + str(delay_mm.shape))    

        iZero_all = np.append(iZero_all, iZero)
        delay_mm_all = np.append(delay_mm_all, delay_mm)
        DataFluo_unpump_norm_total_all = np.append(DataFluo_unpump_norm_total_all, DataFluo_unpump_norm_total)
        DataFluo_pump_norm_total_all = np.append(DataFluo_pump_norm_total_all, DataFluo_pump_norm_total)
        DataFluo_unpump_norm_total_all_err = np.append(DataFluo_unpump_norm_total_all_err, np.square(err_DataFluo_unpump_total))
        DataFluo_pump_norm_total_all_err = np.append(DataFluo_pump_norm_total_all_err, np.square(err_DataFluo_pump_total))


        DataFluo_unpump_norm_total_all_err = 1/np.sqrt(len(scanlist))*np.sqrt(DataFluo_unpump_norm_total_all_err)
        DataFluo_pump_norm_total_all_err = 1/np.sqrt(len(scanlist))*np.sqrt(DataFluo_pump_norm_total_all_err)                                             

        n1 = delay_mm_all.shape[0]
        n2 = len(scanlist)

        delay_mm_all = np.reshape(delay_mm_all, (n2, int(n1/n2)))
        DataFluo_unpump_norm_total_all = np.reshape(DataFluo_unpump_norm_total_all, (n2, int(n1/n2)))
        DataFluo_pump_norm_total_all = np.reshape(DataFluo_pump_norm_total_all, (n2, int(n1/n2)))

        delay_mm_all = np.mean(delay_mm_all, axis=0)
        DataFluo_unpump_norm_total_all = np.mean(DataFluo_unpump_norm_total_all, axis=0)
        DataFluo_pump_norm_total_all = np.mean(DataFluo_pump_norm_total_all, axis=0)
        DataFluo_unpump_norm_total_all_err = np.std(DataFluo_unpump_norm_total_all, axis=0)
        DataFluo_pump_norm_total_all_err = np.std(DataFluo_pump_norm_total_all, axis=0)

        normaliz = np.mean(DataFluo_unpump_norm_total_all)

        diff = (DataFluo_pump_norm_total_all - DataFluo_unpump_norm_total_all)/normaliz
        diff_err = np.sqrt(np.square(DataFluo_pump_norm_total_all_err) + np.square(DataFluo_unpump_norm_total_all_err))\
            /normaliz

        print('Normalization = ' + str(normaliz))


        # convert delays to ps
        #time_zero_mm = 160
        time_zero_mm = 156.3276 #RuDimerACN_timescan_*
        delay_ps = np.array(1e12*(delay_mm_all - time_zero_mm)*2*1e-3/(3e8))

        p = DataFluo_pump_norm_total_all
        u = DataFluo_unpump_norm_total_all
        p_err = DataFluo_pump_norm_total_all_err
        u_err = DataFluo_unpump_norm_total_all_err

    return scan_name, delay_ps, diff, diff_err, p, p_err, u, u_err, iZero_all

In [7]:
ploton = True

lin_filter = 0.1
numstds = 2
minIzero = 0.005

folder_name = "RuDimerACN_timescan_"

scanlist_1 = [24]

DIR_json = "/sf/alvra/data/p17983/res/scan_info/"
scan_name_1, delay_ps_1, diff_1, diff_err_1, p_1, p_err_1, u_1, u_err_1, iZero_all_01 = \
    get_XAS_average_scanlist(numstds, minIzero, folder_name, scanlist_1, DIR_json)

    
scanlist_2 = [30]
scanlist_2 = [27, 28]

DIR_json = "/sf/alvra/data/p17983/res/scan_info/"
scan_name_2, delay_ps_2, diff_2, diff_err_2, p_2, p_err_2, u_2, u_err_2, iZero_all_02 = \
    get_XAS_average_scanlist(numstds, minIzero, folder_name, scanlist_2, DIR_json)
    

Opening /sf/alvra/data/p17983/raw/scan_data/RuDimerACN_timescan_024/...
/sf/alvra/data/p17983/res/scan_info/RuDimerACN_timescan_024_scan_info.json
Normalization = 3.9452285396151248
Opening /sf/alvra/data/p17983/raw/scan_data/RuDimerACN_timescan_027/...
/sf/alvra/data/p17983/res/scan_info/RuDimerACN_timescan_027_scan_info.json


ValueError: cannot reshape array of size 23 into shape (2,11)

In [ ]:
plt.figure()
plt.plot(delay_ps_1, iZero_all_01, label='IZero_01')



plt.figure()
plt.plot(delay_ps_1, p_1/iZero_all_01, label='pumped')
plt.fill_between(delay_ps_1, p_1/iZero_all_01 - p_err_1/iZero_all_01, p_1/iZero_all_01 + p_err_1/iZero_all_01, alpha=0.2) 
plt.plot(delay_ps_1, u_1/iZero_all_01, label='unpumped') 
plt.fill_between(delay_ps_1, u_1/iZero_all_01 - u_err_1/iZero_all_01, u_1/iZero_all_01 + u_err_1/iZero_all_01, alpha = 0.2)
plt.legend()
plt.title('XAS ' + scan_name_1)
plt.ylabel('totally arb')
plt.xlabel('delay (ps)')
plt.grid()


plt.figure()
plt.plot(delay_ps_1, p_1, label='pumped')
plt.fill_between(delay_ps_1, p_1 - p_err_1, p_1 + p_err_1, alpha=0.2) 
plt.plot(delay_ps_1, u_1, label='unpumped') 
plt.fill_between(delay_ps_1, u_1 - u_err_1, u_1 + u_err_1, alpha = 0.2)
plt.legend()
plt.title('XAS ' + scan_name_1)
plt.ylabel('totally arb')
plt.xlabel('delay (ps)')
plt.grid()

plt.figure()
plt.plot(delay_ps_1, diff_1, label='(p - u)/mean(u), ' + scan_name_1)
plt.fill_between(delay_ps_1, diff_1 - diff_err_1, diff_1 + diff_err_1, alpha = 0.2)
plt.legend()
plt.title('XAS ' + scan_name_1)
plt.ylabel('totally arb')
plt.xlabel('delay (ps)')
plt.grid()


In [ ]:
plt.figure()
plt.plot(delay_ps_2, p_2, label='pumped')
plt.fill_between(delay_ps_2, p_2 - p_err_2, p_2 + p_err_2, alpha=0.2) 
plt.plot(delay_ps_2, u_2, label='unpumped') 
plt.fill_between(delay_ps_2, u_2 - u_err_2, u_2 + u_err_2, alpha = 0.2)
plt.legend()
plt.title('XAS ' + scan_name_2)
plt.ylabel('totally arb')
plt.xlabel('delay (ps)')
plt.grid()

plt.figure()
plt.plot(delay_ps_2, diff_2, label='(p - u)/mean(u), ' + scan_name_2)
plt.fill_between(delay_ps_2, diff_2 - diff_err_2, diff_2 + diff_err_2, alpha = 0.2)
plt.legend()
plt.title('XAS ' + scan_name_2)
plt.ylabel('totally arb')
plt.xlabel('delay (ps)')
plt.grid()